<a href="https://colab.research.google.com/github/katha815/ADME-Classifier/blob/main/OData_Features_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Mounted at /content/gdrive


##PhastCons & phyloP features extraction

In [ ]:
# *Check with the early dataset: still three duplicates.
# Other findings: if Gene names are selected for the orginial 337 SNVs data,
# no duplicates are found; However, if all other indentifiers are chosen,
# there are still 3 duplicates.
file_path = f"/content/gdrive/My Drive/Colab Notebooks/ADME Project/features.csv"
df = pd.read_csv(file_path, )
print(df.shape)
df2 = df.drop_duplicates(subset=['chrom',	'pos', 'deleterious', 'ref_allele',	'alt_allele'], keep='first')
# *Shocking results: if include 'deleterious' label than only two duplicates
print(df2.shape)
df_duplicates = df[~df.index.isin(df2.index)]
print(df_duplicates.shape)
print(df_duplicates)

(337, 27)
(335, 27)
(2, 27)
     chrom       pos   Con4   Con7  Con17  Con20  Con30  Con100     P4     P7  \
67   chr10  94942290  0.967  0.251  0.008  0.002  0.018   0.004  0.395 -1.132   
234  chr22  42130692  0.913  0.321  0.082  0.255  0.448   1.000  0.543  0.917   

     ...  alt_allele  baseChange  deleterious  3_prime_UTR_variant  \
67   ...           T           6            1                    0   
234  ...           T           9            1                    0   

     *missense_variant  NMD_transcript_variant *synonymous_variant  \
67                   1                       1                   0   
234                  1                       0                   0   

    *stop_gained  5_prime_UTR_variant  *start_lost  
67             0                    0            0  
234            0                    0            0  

[2 rows x 27 columns]


In [ ]:
# Filter the 'features' dataset by deleting duplicates and keep the deleterious-labelled data
filterd_df0 = df[~((df.duplicated(subset=['chrom',	'pos','ref_allele',	'alt_allele']) & (df['deleterious'] == 0)) |
      ((df['deleterious'] == 1) & df.duplicated(subset=['chrom',	'pos','ref_allele',	'alt_allele']) & (df.groupby(by=['chrom',	'pos','ref_allele',	'alt_allele'])['deleterious'].transform('max') == 1)))]
print(filterd_df0.shape)

(334, 27)


In [ ]:
num = [4, 7, 17 ,20, 30, 100]
pd_Con = []
for x in num:
    file_path = f"/content/gdrive/My Drive/Colab Notebooks/ADME Project/Ori_phastCons{x}way.bedgraph"
    df = pd.read_csv(file_path, delimiter='\t',header=None)
    # Specify the column names
    column_names = ['Chromo',	'Location1',	'Location2',	'Ref',	'Alter',	'Label',	'chr',	'Gene',	'Match1',	'Match2',	'Match3',	'Match4',	'Score']

    # Assign the column names to the DataFrame
    df.columns = column_names

    # Delete duplicates1
    df = df.drop_duplicates(subset=['Chromo',	'Location1',	'Location2',	'Ref',	'Alter',	'Label', 'chr',	'Gene'], keep='first')

    # Delete duplicates2
    filterd_df = df[~((df.duplicated(subset=['Chromo','Location1','Ref',	'Alter']) & (df['Label'] == 0)) |
      ((df['Label'] == 1) & df.duplicated(subset=['Chromo','Location1','Ref',	'Alter']) & (df.groupby(by=['Chromo','Location1','Ref',	'Alter'])['Label'].transform('max') == 1)))]

    # Sorting
    filterd_df_sorted = filterd_df.sort_values(by=['Chromo',	'Location1','Alter'])

    pd_Con.append(filterd_df_sorted)

# Access the individual DataFrames in the list
for i, df in enumerate(pd_Con):
    print(f"DataFrame {i+1}:")
    print(df.head(3))
    # print(df)

DataFrame 1:
  Chromo  Location1  Location2 Ref Alter  Label  chr     Gene Match1  \
0   chr1   46930150   46930150   C     T      1    1  CYP4A11   chr1   
2   chr1   46930161   46930161   G     A      1    1  CYP4A11   chr1   
4   chr1   46932824   46932824   T     C      1    1  CYP4A11   chr1   

     Match2    Match3       Match4  Score  
0  46930149  46930150  id-31018927  0.014  
2  46930160  46930161  id-31018938  0.014  
4  46932823  46932824  id-31021601  0.823  
DataFrame 2:
  Chromo  Location1  Location2 Ref Alter  Label  chr     Gene Match1  \
0   chr1   46930150   46930150   C     T      1    1  CYP4A11   chr1   
2   chr1   46930161   46930161   G     A      1    1  CYP4A11   chr1   
4   chr1   46932824   46932824   T     C      1    1  CYP4A11   chr1   

     Match2    Match3       Match4  Score  
0  46930149  46930150  id-45570473  0.007  
2  46930160  46930161  id-45570484  0.063  
4  46932823  46932824  id-45573147  0.104  
DataFrame 3:
  Chromo  Location1  Location2 

In [ ]:
pd_Con[0]

,Chromo,Location1,Location2,Ref,Alter,Label,chr,Gene,Match1,Match2,Match3,Match4,Score
0,chr1,46930150,46930150,C,T,1,1,CYP4A11,chr1,46930149,46930150,id-31018927,0.014
2,chr1,46930161,46930161,G,A,1,1,CYP4A11,chr1,46930160,46930161,id-31018938,0.014
4,chr1,46932824,46932824,T,C,1,1,CYP4A11,chr1,46932823,46932824,id-31021601,0.823
6,chr1,46932987,46932987,C,T,1,1,CYP4A11,chr1,46932986,46932987,id-31021764,0.982
8,chr1,46934207,46934207,A,G,0,1,CYP4A11,chr1,46934206,46934207,id-31022978,0.068
...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,chr8,104466712,104466712,T,C,1,8,DPYS,chr8,104466711,104466712,id-1705230967,0.998
666,chr8,104466904,104466904,G,A,0,8,DPYS,chr8,104466903,104466904,id-1705231159,0.326
668,chr8,142875730,142875730,C,A,1,8,CYP11B1,chr8,142875729,142875730,id-1733168123,0.347
670,chr8,142879146,142879146,C,T,1,8,CYP11B1,chr8,142879145,142879146,id-1733171538,0.366


In [ ]:
df1_sorted = pd_Con[0].sort_values(by=['Chromo',	'Location1','Alter'])
filterd_df_sorted0 = filterd_df0.sort_values(by=['chrom',	'pos','alt_allele'])
pd_check = pd.merge_ordered(filterd_df_sorted0 , df1_sorted,left_on=['chrom',	'pos','alt_allele'],right_on=['Chromo',	'Location1','Alter'],how='left',fill_method="ffill")
print(pd_check.shape)
pd_check

(334, 40)


,chrom,pos,Con4,Con7,Con17,Con20,Con30,Con100,P4,P7,...,Ref,Alter,Label,chr,Gene,Match1,Match2,Match3,Match4,Score
0,chr1,46930150,0.014,0.007,0.019,0.049,0.088,0.896,-0.330,-0.821,...,C,T,1,1,CYP4A11,chr1,46930149,46930150,id-31018927,0.014
1,chr1,46930161,0.014,0.063,0.000,0.010,0.010,0.000,-0.854,0.061,...,G,A,1,1,CYP4A11,chr1,46930160,46930161,id-31018938,0.014
2,chr1,46932824,0.823,0.104,0.104,0.440,0.329,0.000,-0.280,-0.597,...,T,C,1,1,CYP4A11,chr1,46932823,46932824,id-31021601,0.823
3,chr1,46932987,0.982,0.962,0.078,0.962,0.996,1.000,0.606,0.917,...,C,T,1,1,CYP4A11,chr1,46932986,46932987,id-31021764,0.982
4,chr1,46934207,0.068,0.017,0.000,0.000,0.001,0.000,0.669,-0.328,...,A,G,0,1,CYP4A11,chr1,46934206,46934207,id-31022978,0.068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,chr8,104466712,0.998,1.000,0.989,0.999,1.000,1.000,0.714,1.039,...,T,C,1,8,DPYS,chr8,104466711,104466712,id-1705230967,0.998
330,chr8,104466904,0.326,0.561,0.011,0.003,0.006,0.253,0.575,0.853,...,G,A,0,8,DPYS,chr8,104466903,104466904,id-1705231159,0.326
331,chr8,142875730,0.347,0.994,0.282,0.863,0.461,1.000,0.606,0.917,...,C,A,1,8,CYP11B1,chr8,142875729,142875730,id-1733168123,0.347
332,chr8,142879146,0.366,0.229,0.131,0.192,0.067,1.000,0.606,0.917,...,C,T,1,8,CYP11B1,chr8,142879145,142879146,id-1733171538,0.366


In [ ]:
merged_df = pd_Con[0].iloc[:, :7]

# Merge "Score" columns from all DataFrames
for i, df in enumerate(pd_Con):
    column_name = f"Con{num[i]}"
    merged_df[column_name] = df["Score"]

# Print the merged DataFrame
merged_df

,Chromo,Location1,Location2,Ref,Alter,Label,chr,Con4,Con7,Con17,Con20,Con30,Con100
0,chr1,46930150,46930150,C,T,1,1,0.014,0.007,0.019,0.049,0.088,0.896
2,chr1,46930161,46930161,G,A,1,1,0.014,0.063,0.000,0.010,0.010,0.000
4,chr1,46932824,46932824,T,C,1,1,0.823,0.104,0.104,0.440,0.329,0.000
6,chr1,46932987,46932987,C,T,1,1,0.982,0.962,0.078,0.962,0.996,1.000
8,chr1,46934207,46934207,A,G,0,1,0.068,0.017,0.000,0.000,0.001,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,chr8,104466712,104466712,T,C,1,8,0.998,1.000,0.989,0.999,1.000,1.000
666,chr8,104466904,104466904,G,A,0,8,0.326,0.561,0.011,0.003,0.006,0.253
668,chr8,142875730,142875730,C,A,1,8,0.347,0.994,0.282,0.863,0.461,1.000
670,chr8,142879146,142879146,C,T,1,8,0.366,0.229,0.131,0.192,0.067,1.000


In [ ]:
pd_P = []
for x in num:
    file_path = f"/content/gdrive/My Drive/Colab Notebooks/ADME Project/Ori_phyloP{x}way.bedgraph"
    df = pd.read_csv(file_path, delimiter='\t',header=None)
    # Specify the column names
    column_names = ['Chromo',	'Location1',	'Location2',	'Ref',	'Alter',	'Label',	'chr',	'Gene',	'Match1',	'Match2',	'Match3',	'Match4',	'Score']

    # Assign the column names to the DataFrame
    df.columns = column_names

    # Delete duplicates1
    df = df.drop_duplicates(subset=['Chromo',	'Location1',	'Location2',	'Ref',	'Alter',	'Label', 'chr',	'Gene'], keep='first')

    # Delete duplicates2
    filterd_df = df[~((df.duplicated(subset=['Chromo','Location1','Ref',	'Alter']) & (df['Label'] == 0)) |
      ((df['Label'] == 1) & df.duplicated(subset=['Chromo','Location1','Ref',	'Alter']) & (df.groupby(by=['Chromo','Location1','Ref',	'Alter'])['Label'].transform('max') == 1)))]

    # Sorting
    filterd_df_sorted = filterd_df.sort_values(by=['Chromo',	'Location1','Alter'])

    pd_P.append(filterd_df_sorted)

# Access the individual DataFrames in the list
for i, df in enumerate(pd_P):
    print(f"DataFrame {i+1}:", df.shape)
    print(df.head(3))
    # print(df)

DataFrame 1: (334, 13)
  Chromo  Location1  Location2 Ref Alter  Label  chr     Gene Match1  \
0   chr1   46930150   46930150   C     T      1    1  CYP4A11   chr1   
2   chr1   46930161   46930161   G     A      1    1  CYP4A11   chr1   
4   chr1   46932824   46932824   T     C      1    1  CYP4A11   chr1   

     Match2    Match3       Match4  Score  
0  46930149  46930150  id-31018927 -0.330  
2  46930160  46930161  id-31018938 -0.854  
4  46932823  46932824  id-31021601 -0.280  
DataFrame 2: (334, 13)
  Chromo  Location1  Location2 Ref Alter  Label  chr     Gene Match1  \
0   chr1   46930150   46930150   C     T      1    1  CYP4A11   chr1   
2   chr1   46930161   46930161   G     A      1    1  CYP4A11   chr1   
4   chr1   46932824   46932824   T     C      1    1  CYP4A11   chr1   

     Match2    Match3       Match4  Score  
0  46930149  46930150  id-45570473 -0.821  
2  46930160  46930161  id-45570484  0.061  
4  46932823  46932824  id-45573147 -0.597  
DataFrame 3: (334, 13)
 

In [ ]:
for i, df in enumerate(pd_P):
    column_name = f"P{num[i]}"
    merged_df[column_name] = df["Score"]

# Print the merged DataFrame
merged_df

,Chromo,Location1,Location2,Ref,Alter,Label,chr,Con4,Con7,Con17,Con20,Con30,Con100,P4,P7,P17,P20,P30,P100
0,chr1,46930150,46930150,C,T,1,1,0.014,0.007,0.019,0.049,0.088,0.896,-0.330,-0.821,0.674,0.078,0.166,1.555
2,chr1,46930161,46930161,G,A,1,1,0.014,0.063,0.000,0.010,0.010,0.000,-0.854,0.061,-4.106,-2.440,-2.350,-0.585
4,chr1,46932824,46932824,T,C,1,1,0.823,0.104,0.104,0.440,0.329,0.000,-0.280,-0.597,-0.685,-0.237,-0.545,0.076
6,chr1,46932987,46932987,C,T,1,1,0.982,0.962,0.078,0.962,0.996,1.000,0.606,0.917,0.667,1.040,1.168,6.984
8,chr1,46934207,46934207,A,G,0,1,0.068,0.017,0.000,0.000,0.001,0.000,0.669,-0.328,-2.722,-1.360,-0.609,-0.081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,chr8,104466712,104466712,T,C,1,8,0.998,1.000,0.989,0.999,1.000,1.000,0.714,1.039,0.584,1.051,1.223,8.686
666,chr8,104466904,104466904,G,A,0,8,0.326,0.561,0.011,0.003,0.006,0.253,0.575,0.853,-0.942,-0.493,0.127,0.776
668,chr8,142875730,142875730,C,A,1,8,0.347,0.994,0.282,0.863,0.461,1.000,0.606,0.917,0.646,1.022,0.957,5.422
670,chr8,142879146,142879146,C,T,1,8,0.366,0.229,0.131,0.192,0.067,1.000,0.606,0.917,0.613,0.993,0.889,3.627


In [ ]:
merged_df[['chr', 'Location1']].dtypes

chr          int64
Location1    int64
dtype: object

## Ensembl Features Extraction

In [ ]:
pd_En = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/ADME Project/Odata_Ensembl.txt", delimiter='\t')
pd_En.replace('-', np.nan, inplace=True)
pd_En

,#Uploaded_variation,Location,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,...,gnomADg_SAS_AF,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS
0,1_46930150_C/T,1:46930150-46930150,T,missense_variant,MODERATE,CYP4A11,ENSG00000187048,Transcript,ENST00000310638.9,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1_46930150_C/T,1:46930150-46930150,T,downstream_gene_variant,MODIFIER,CYP4A11,ENSG00000187048,Transcript,ENST00000371905.1,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1_46930150_C/T,1:46930150-46930150,T,missense_variant,MODERATE,CYP4A11,ENSG00000187048,Transcript,ENST00000462347.5,protein_coding,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1_46930150_C/T,1:46930150-46930150,T,downstream_gene_variant,MODIFIER,CYP4A11,ENSG00000187048,Transcript,ENST00000465874.5,nonsense_mediated_decay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1_46930150_C/T,1:46930150-46930150,T,"3_prime_UTR_variant,NMD_transcript_variant",MODIFIER,CYP4A11,ENSG00000187048,Transcript,ENST00000468629.5,nonsense_mediated_decay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270,9_84279999_T/C,9:84279999-84279999,C,missense_variant,MODERATE,SLC28A3,64078,Transcript,XM_011518905.3,protein_coding,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4271,9_84279999_T/C,9:84279999-84279999,C,missense_variant,MODERATE,SLC28A3,64078,Transcript,XM_011518906.3,protein_coding,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4272,9_84279999_T/C,9:84279999-84279999,C,missense_variant,MODERATE,SLC28A3,64078,Transcript,XM_011518907.3,protein_coding,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4273,9_84279999_T/C,9:84279999-84279999,C,missense_variant,MODERATE,SLC28A3,64078,Transcript,XM_011518908.3,protein_coding,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd_En.dtypes

#Uploaded_variation      object
Location                 object
Allele                   object
Consequence              object
IMPACT                   object
                          ...  
MOTIF_NAME               object
MOTIF_POS                object
HIGH_INF_POS             object
MOTIF_SCORE_CHANGE       object
TRANSCRIPTION_FACTORS    object
Length: 63, dtype: object

In [ ]:
columns_to_convert = ['AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF']

# Convert the specified columns to float type
pd_En[columns_to_convert] = pd_En[columns_to_convert].astype(float)
pd_En.dtypes

#Uploaded_variation      object
Location                 object
Allele                   object
Consequence              object
IMPACT                   object
                          ...  
MOTIF_NAME               object
MOTIF_POS                object
HIGH_INF_POS             object
MOTIF_SCORE_CHANGE       object
TRANSCRIPTION_FACTORS    object
Length: 63, dtype: object

In [ ]:
pd_En2 = pd_En
# Extract first number after ":"
pd_En2['Location'] = pd_En['Location'].str.extract(r'\:(\d+)')

# Extract number inside brackets in "SIFT" column
pd_En2['SIFT'] = pd_En['SIFT'].str.extract(r'\((\d+\.\d+)\)').astype(float)

# Extract number inside brackets in "PolyPhen" column
pd_En2['PolyPhen'] = pd_En['PolyPhen'].str.extract(r'\((\d+\.\d+)\)').astype(float)

#DISTANCE
pd_En2['DISTANCE'] = pd_En['DISTANCE'].str.extract(r'(\d+)').astype(float)

# Print the modified DataFrame
pd_En2[['Location','SIFT','PolyPhen','DISTANCE']]

,Location,SIFT,PolyPhen,DISTANCE
0,46930150,0.29,0.167,NaN
1,46930150,NaN,NaN,2281.0
2,46930150,0.17,0.081,NaN
3,46930150,NaN,NaN,1356.0
4,46930150,NaN,NaN,NaN
...,...,...,...,...
4270,84279999,NaN,NaN,NaN
4271,84279999,NaN,NaN,NaN
4272,84279999,NaN,NaN,NaN
4273,84279999,NaN,NaN,NaN


In [ ]:
#1: Calculate max, min and mean for interested number columns
aggregations = {
    'DISTANCE': ['max', 'min', 'mean'],
    'SIFT': ['max', 'min', 'mean'],
    'PolyPhen': ['max', 'min', 'mean'],
}
grouped_df = pd_En2.groupby('#Uploaded_variation').agg(aggregations)
mean_df = pd_En2.groupby('#Uploaded_variation')[columns_to_convert].agg('mean')
grouped_df

DISTANCE                 SIFT                 PolyPhen  \
                         max    min    mean   max   min      mean      max   
#Uploaded_variation                                                          
10_102834967_C/T      3264.0  451.0  1668.2  0.14  0.03  0.114000    0.985   
10_94762713_C/A          NaN    NaN     NaN  0.07  0.01  0.050000    0.943   
10_94762751_C/T          NaN    NaN     NaN  0.15  0.13  0.143333    0.166   
10_94762755_T/C          NaN    NaN     NaN  0.01  0.01  0.010000    0.246   
10_94762760_A/C          NaN    NaN     NaN   NaN   NaN       NaN      NaN   
...                      ...    ...     ...   ...   ...       ...      ...   
8_18400502_G/A           NaN    NaN     NaN   NaN   NaN       NaN    0.007   
8_18400593_G/A           NaN    NaN     NaN  0.06  0.04  0.045000    0.995   
8_18400848_A/C           NaN    NaN     NaN  0.10  0.03  0.082500    0.998   
8_18400860_G/A           NaN    NaN     NaN  0.63  0.51  0.540000    0.047   
9_84279999_T/C           NaN    NaN     NaN  0.01  0.01  0.010000    0.288   

                                    
                       min    mean  
#Uploaded_variation                 
10_102834967_C/T     0.049  0.3444  
10_94762713_C/A      0.883  0.9230  
10_94762751_C/T      0.166  0.1660  
10_94762755_T/C      0.168  0.2200  
10_94762760_A/C        NaN     NaN  
...                    ...     ...  
8_18400502_G/A       0.007  0.0070  
8_18400593_G/A       0.995  0.9950  
8_18400848_A/C       0.998  0.9980  
8_18400860_G/A       0.047  0.0470  
9_84279999_T/C       0.288  0.2880  

[334 rows x 9 columns]

In [ ]:
mean_df

,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF
#Uploaded_variation,,,,,,,,,,,,,,,,,
10_102834967_C/T,NaN,NaN,NaN,NaN,NaN,NaN,0.000013,0.000024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000015,0.00000,0.000000
10_94762713_C/A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10_94762751_C/T,NaN,NaN,NaN,NaN,NaN,NaN,0.000007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000015,0.00000,0.000000
10_94762755_T/C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10_94762760_A/C,0.0046,0.0166,0.0000,0.0000,0.0000,0.0010,0.005733,0.018900,0.000000,0.002225,0.004037,0.000000,0.000000,0.003185,0.000441,0.00382,0.000415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8_18400502_G/A,0.0002,0.0000,0.0000,0.0010,0.0000,0.0000,0.000118,0.000000,0.000000,0.000000,0.000865,0.000968,0.000568,0.000000,0.000044,0.00000,0.000208
8_18400593_G/A,0.2650,0.2375,0.1715,0.2560,0.2823,0.3599,0.273300,0.258700,0.353100,0.207100,0.361500,0.256100,0.234100,0.286600,0.292100,0.27370,0.365900
8_18400848_A/C,NaN,NaN,NaN,NaN,NaN,NaN,0.000013,0.000000,0.000000,0.000065,0.000000,0.000000,0.000000,0.000000,0.000015,0.00000,0.000000


In [ ]:
# New data frame without the columns to be converted
pd_En3 = pd_En2
pd_En3 = pd_En3.drop(columns=['Location','SIFT','PolyPhen','DISTANCE'])
pd_En3 = pd_En3.drop(columns=['Consequence','IMPACT','BIOTYPE','FLAGS'])
pd_En3 = pd_En3.drop(columns=columns_to_convert)
pd_En3

,#Uploaded_variation,Allele,SYMBOL,Gene,Feature_type,Feature,EXON,INTRON,HGVSc,HGVSp,...,BAM_EDIT,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS
0,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000310638.9,12/12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000371905.1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000462347.5,10/10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000465874.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000468629.5,11/11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518905.3,16/18,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4271,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518906.3,17/19,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4272,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518907.3,15/17,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
4273,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518908.3,11/13,NaN,NaN,NaN,...,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#1: Calculate max, min and mean for interested number columns
aggregations = {
    'DISTANCE': ['max', 'min', 'mean'],
    'SIFT': ['max', 'min', 'mean'],
    'PolyPhen': ['max', 'min', 'mean'],
}
grouped_df = pd_En2.groupby('#Uploaded_variation').agg(aggregations)
mean_df = pd_En2.groupby('#Uploaded_variation')[columns_to_convert].mean().reset_index()

# Merge the aggregated data with the original DataFrame
pd_En3 = pd_En3.merge(grouped_df, on='#Uploaded_variation')
pd_En3 = pd_En3.merge(mean_df, on='#Uploaded_variation')
pd_En3

<ipython-input-19-b3b125a19d88>:11: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  pd_En3 = pd_En3.merge(grouped_df, on='#Uploaded_variation')


,#Uploaded_variation,Allele,SYMBOL,Gene,Feature_type,Feature,EXON,INTRON,HGVSc,HGVSp,...,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF
0,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000310638.9,12/12,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000371905.1,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000462347.5,10/10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000465874.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000468629.5,11/11,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518905.3,16/18,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4271,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518906.3,17/19,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4272,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518907.3,15/17,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4273,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518908.3,11/13,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#2: Combine text data from columns to create a new column
pd_En3['Consequence'] = pd_En2['Consequence'].str.split(',')
# Define the list of terms
terms = [
    "transcript_ablation",
    "splice_acceptor_variant",
    "splice_donor_variant",
    "stop_gained",
    "frameshift_variant",
    "stop_lost",
    "start_lost",
    "transcript_amplification",
    "feature_elongation",
    "feature_truncation",
    "inframe_insertion",
    "inframe_deletion",
    "missense_variant",
    "protein_altering_variant",
    "splice_donor_5th_base_variant",
    "splice_region_variant",
    "splice_donor_region_variant",
    "splice_polypyrimidine_tract_variant",
    "incomplete_terminal_codon_variant",
    "start_retained_variant",
    "stop_retained_variant",
    "synonymous_variant",
    "coding_sequence_variant",
    "mature_miRNA_variant",
    "5_prime_UTR_variant",
    "3_prime_UTR_variant",
    "non_coding_transcript_exon_variant",
    "intron_variant",
    "NMD_transcript_variant",
    "non_coding_transcript_variant",
    "coding_transcript_variant",
    "upstream_gene_variant",
    "downstream_gene_variant",
    "TFBS_ablation",
    "TFBS_amplification",
    "TF_binding_site_variant",
    "regulatory_region_ablation",
    "regulatory_region_amplification",
    "regulatory_region_variant",
    "intergenic_variant",
    "sequence_variant"
]


# Create new columns and set initial values to 0
for term in terms:
    pd_En3[term] = 0

# Iterate over each row and update the corresponding columns
for index, row in pd_En3.iterrows():
    variant_list = row['Consequence']
    for variant in variant_list:
        if variant in terms:
            pd_En3.at[index, variant] = 1

# Drop the 'variants' column if needed
pd_En3 = pd_En3.drop('Consequence', axis=1)
# One-hot encode the new column
#one_hot_encoded1 = pd.get_dummies(pd_En3['Consequence'].apply(pd.Series).stack()).sum(level=0)
# one_hot_encoded2 = pd.get_dummies(pd_En3['Amino_acids'])
# one_hot_encoded3 = pd.get_dummies(pd_En3['Codons'])
pd_En3['FLAGS'] = pd_En2['FLAGS'].str.split(',')
one_hot_encoded4 = pd.get_dummies(pd_En3['FLAGS'].apply(pd.Series).stack()).sum(level=0)
pd_En3.drop(columns=['FLAGS'])
# Merge the one-hot encoded columns with the original DataFrame
pd_En3 = pd.concat([pd_En3, #one_hot_encoded1,#one_hot_encoded2,one_hot_encoded3,
              one_hot_encoded4], axis=1)

# Print the modified DataFrame
# pd_En3.to_csv('/content/gdrive/My Drive/Colab Notebooks/ADME Project/Odata_pd_En3.csv', index=False)
pd_En3

<ipython-input-20-58b0f3eeec80>:67: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  one_hot_encoded4 = pd.get_dummies(pd_En3['FLAGS'].apply(pd.Series).stack()).sum(level=0)


,#Uploaded_variation,Allele,SYMBOL,Gene,Feature_type,Feature,EXON,INTRON,HGVSc,HGVSp,...,TFBS_amplification,TF_binding_site_variant,regulatory_region_ablation,regulatory_region_amplification,regulatory_region_variant,intergenic_variant,sequence_variant,FLAGS,cds_end_NF,cds_start_NF
0,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000310638.9,12/12,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000371905.1,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000462347.5,10/10,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000465874.5,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1_46930150_C/T,T,CYP4A11,ENSG00000187048,Transcript,ENST00000468629.5,11/11,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518905.3,16/18,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4271,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518906.3,17/19,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4272,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518907.3,15/17,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4273,9_84279999_T/C,C,SLC28A3,64078,Transcript,XM_011518908.3,11/13,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
pd_En4 = pd_En3
columns = terms+one_hot_encoded4.columns.tolist()
      #one_hot_encoded1.columns.tolist() \
      # one_hot_encoded2.columns.tolist()\+
      #  one_hot_encoded3.columns.tolist()\+

pd_En4 = pd_En4.groupby("#Uploaded_variation")[columns].max().reset_index()
pd_En4 = pd_En4.merge(grouped_df, on='#Uploaded_variation')
pd_En4 = pd_En4.merge(mean_df, on='#Uploaded_variation')
pd_En4.to_csv('/content/gdrive/My Drive/Colab Notebooks/ADME Project/Odata_pd_En4.csv')
pd_En4

<ipython-input-21-5a6115f3be64>:8: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  pd_En4 = pd_En4.merge(grouped_df, on='#Uploaded_variation')


,#Uploaded_variation,transcript_ablation,splice_acceptor_variant,splice_donor_variant,stop_gained,frameshift_variant,stop_lost,start_lost,transcript_amplification,feature_elongation,...,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF
0,10_102834967_C/T,0,0,0,0,0,0,0,0,0,...,0.000024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000015,0.00000,0.000000
1,10_94762713_C/A,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10_94762751_C/T,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000015,0.00000,0.000000
3,10_94762755_T/C,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10_94762760_A/C,0,0,0,0,0,0,0,0,0,...,0.018900,0.000000,0.002225,0.004037,0.000000,0.000000,0.003185,0.000441,0.00382,0.000415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,8_18400502_G/A,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000865,0.000968,0.000568,0.000000,0.000044,0.00000,0.000208
330,8_18400593_G/A,0,0,0,0,0,0,0,0,0,...,0.258700,0.353100,0.207100,0.361500,0.256100,0.234100,0.286600,0.292100,0.27370,0.365900
331,8_18400848_A/C,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000065,0.000000,0.000000,0.000000,0.000000,0.000015,0.00000,0.000000
332,8_18400860_G/A,0,0,0,0,0,0,0,0,0,...,0.033100,0.002193,0.095820,0.020470,0.150900,0.033960,0.047770,0.025740,0.05455,0.073860


In [ ]:
pd_En5 = pd.DataFrame(pd_En4)
pd_En5[['chr', 'Location1', 'baseChange']] = pd_En4['#Uploaded_variation'].str.split('_', expand=True)
pd_En5[['Ref', 'Alter']] = pd_En5['baseChange'].str.split('/', expand=True)
pd_En5.drop('#Uploaded_variation', axis=1, inplace=True)
pd_En5[['chr', 'Location1']] = pd_En5[['chr', 'Location1']].astype(int)
pd_En5

,transcript_ablation,splice_acceptor_variant,splice_donor_variant,stop_gained,frameshift_variant,stop_lost,start_lost,transcript_amplification,feature_elongation,feature_truncation,...,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,chr,Location1,baseChange,Ref,Alter
0,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000015,0.00000,0.000000,10,102834967,C/T,C,T
1,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,10,94762713,C/A,C,A
2,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000015,0.00000,0.000000,10,94762751,C/T,C,T
3,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,10,94762755,T/C,T,C
4,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.003185,0.000441,0.00382,0.000415,10,94762760,A/C,A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,0,0,0,0,0,0,0,0,0,0,...,0.000568,0.000000,0.000044,0.00000,0.000208,8,18400502,G/A,G,A
330,0,0,0,0,0,0,0,0,0,0,...,0.234100,0.286600,0.292100,0.27370,0.365900,8,18400593,G/A,G,A
331,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.000015,0.00000,0.000000,8,18400848,A/C,A,C
332,0,0,0,0,0,0,0,0,0,0,...,0.033960,0.047770,0.025740,0.05455,0.073860,8,18400860,G/A,G,A


In [ ]:
sorting_col = ['chr','Location1','Ref','Alter']
pd_En5_sorted = pd_En5.sort_values(by=sorting_col)
merged_df_sorted = merged_df.sort_values(by=sorting_col)

In [ ]:
pd_En5_sorted.head()

,transcript_ablation,splice_acceptor_variant,splice_donor_variant,stop_gained,frameshift_variant,stop_lost,start_lost,transcript_amplification,feature_elongation,feature_truncation,...,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,chr,Location1,baseChange,Ref,Alter
178,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1,46930150,C/T,C,T
179,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1,46930161,G/A,G,A
180,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1,46932824,T/C,T,C
181,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1,46932987,C/T,C,T
182,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,1,46934207,A/G,A,G


In [ ]:
merged_df_sorted.head()

,Chromo,Location1,Location2,Ref,Alter,Label,chr,Con4,Con7,Con17,Con20,Con30,Con100,P4,P7,P17,P20,P30,P100
0,chr1,46930150,46930150,C,T,1,1,0.014,0.007,0.019,0.049,0.088,0.896,-0.330,-0.821,0.674,0.078,0.166,1.555
2,chr1,46930161,46930161,G,A,1,1,0.014,0.063,0.000,0.010,0.010,0.000,-0.854,0.061,-4.106,-2.440,-2.350,-0.585
4,chr1,46932824,46932824,T,C,1,1,0.823,0.104,0.104,0.440,0.329,0.000,-0.280,-0.597,-0.685,-0.237,-0.545,0.076
6,chr1,46932987,46932987,C,T,1,1,0.982,0.962,0.078,0.962,0.996,1.000,0.606,0.917,0.667,1.040,1.168,6.984
8,chr1,46934207,46934207,A,G,0,1,0.068,0.017,0.000,0.000,0.001,0.000,0.669,-0.328,-2.722,-1.360,-0.609,-0.081


In [ ]:
pd_En6 = pd.merge_ordered(merged_df_sorted,pd_En5_sorted,left_on=sorting_col,right_on=sorting_col,how='left',fill_method="ffill")

## Move the conserv scores to the last to match with Sdata and Cdata
# Define the column order with the target columns moved to the end
target_columns = ['Con4', 'Con7', 'Con17', 'Con20', 'Con30', 'Con100', 'P4', 'P7', 'P17', 'P20', 'P30', 'P100']
new_column_order = [col for col in pd_En6.columns if col not in target_columns] + target_columns
# Reindex the DataFrame with the new column order
pd_En6 = pd_En6.reindex(columns=new_column_order)

pd_En6.head()

,Chromo,Location1,Location2,Ref,Alter,Label,chr,transcript_ablation,splice_acceptor_variant,splice_donor_variant,...,Con17,Con20,Con30,Con100,P4,P7,P17,P20,P30,P100
0,chr1,46930150,46930150,C,T,1,1,0,0,0,...,0.019,0.049,0.088,0.896,-0.330,-0.821,0.674,0.078,0.166,1.555
1,chr1,46930161,46930161,G,A,1,1,0,0,0,...,0.000,0.010,0.010,0.000,-0.854,0.061,-4.106,-2.440,-2.350,-0.585
2,chr1,46932824,46932824,T,C,1,1,0,0,0,...,0.104,0.440,0.329,0.000,-0.280,-0.597,-0.685,-0.237,-0.545,0.076
3,chr1,46932987,46932987,C,T,1,1,0,0,0,...,0.078,0.962,0.996,1.000,0.606,0.917,0.667,1.040,1.168,6.984
4,chr1,46934207,46934207,A,G,0,1,0,0,0,...,0.000,0.000,0.001,0.000,0.669,-0.328,-2.722,-1.360,-0.609,-0.081


In [ ]:
# Check for duplicates
pd_En7 = pd_En6.drop_duplicates(subset=sorting_col, keep='first')
print(pd_En7.shape)
df_duplicates = pd_En6[~pd_En6.index.isin(pd_En7.index)]
print(df_duplicates.shape)
print(df_duplicates)

(334, 89)
(0, 89)
Empty DataFrame
Columns: [Chromo, Location1, Location2, Ref, Alter, Label, chr, transcript_ablation, splice_acceptor_variant, splice_donor_variant, stop_gained, frameshift_variant, stop_lost, start_lost, transcript_amplification, feature_elongation, feature_truncation, inframe_insertion, inframe_deletion, missense_variant, protein_altering_variant, splice_donor_5th_base_variant, splice_region_variant, splice_donor_region_variant, splice_polypyrimidine_tract_variant, incomplete_terminal_codon_variant, start_retained_variant, stop_retained_variant, synonymous_variant, coding_sequence_variant, mature_miRNA_variant, 5_prime_UTR_variant, 3_prime_UTR_variant, non_coding_transcript_exon_variant, intron_variant, NMD_transcript_variant, non_coding_transcript_variant, coding_transcript_variant, upstream_gene_variant, downstream_gene_variant, TFBS_ablation, TFBS_amplification, TF_binding_site_variant, regulatory_region_ablation, regulatory_region_amplification, regulatory_region

## Add 'baseChange' feature

In [ ]:
pd_all = pd.concat((pd_En7,pd.get_dummies(pd_En7.baseChange)),1)
pd_all = pd_all.drop('baseChange', axis=1)
pd_all
pd_all.to_csv('/content/gdrive/My Drive/Colab Notebooks/ADME Project/Odata_nof.csv', index=False)

<ipython-input-28-c55c2b616b52>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  pd_all = pd.concat((pd_En7,pd.get_dummies(pd_En7.baseChange)),1)


In [ ]:
# Fill NaN with zero
df_filled_zero = pd_all.fillna(0)
df_filled_zero.to_csv('/content/gdrive/My Drive/Colab Notebooks/ADME Project/Odata_fzero.csv', index=False)
# Fill NaN with mean
df_filled_mean = pd_all.fillna(pd_all.mean())
df_filled_zero.to_csv('/content/gdrive/My Drive/Colab Notebooks/ADME Project/Odata_fmean.csv', index=False)

<ipython-input-29-8d519536af70>:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_filled_mean = pd_all.fillna(pd_all.mean())
